In [2]:
import formulas
import utilities
import pandas as pd
import os
import re


def create_formulas_content(website_creator_dir, docs_dir):

    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formula_sheet_list = formulas.get_formulas_on_formula_sheet(
        formulas_input_df)
    
    by_year_df =  (
        formulas.get_formulas_by_year_df(formulas_input_df))
    cumulative_df = (
        formulas.get_formulas_by_year_cumulative_df(
            formulas_input_df, sort_orders_df))
    
    formulas_df = pd.concat([by_year_df, cumulative_df])

    dirs_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Formula_1', 'Formula_2'])
                          

    calculus_summary_file_paths_df = (
        formulas.calculus_summary_file_paths_df(cumulative_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= calculus_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_calculus_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Differentiation', 'Integration'])
    


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(website_creator_dir, docs_dir)
